# Method 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install langchain
!pip install faiss-cpu
!pip install transformers
!pip install auto_gptq
!pip install -qqq InstructorEmbedding==1.0.1
!pip install pypdf
!pip install sentence_transformers

In [2]:
!pip install -U langchain-community

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from transformers import AutoTokenizer, TextStreamer, pipeline
from auto_gptq import AutoGPTQForCausalLM
from langchain import HuggingFacePipeline, PromptTemplate
import torch

In [4]:
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

DATA_PATH = '/content/drive/MyDrive/chatbot_data/data'
# DATA_PATH = '/content'
DB_FAISS_PATH = 'vectorstore/db_faiss'

# Create vector database
def create_vector_db():
    loader = DirectoryLoader(DATA_PATH,
                             glob='*.pdf',
                             loader_cls=PyPDFLoader)

    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                                   chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    # embeddings = HuggingFaceInstructEmbeddings(
    # model_name="hkunlp/instructor-large", model_kwargs={"device": DEVICE}
# )

    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                       model_kwargs={'device': DEVICE})

    db = FAISS.from_documents(texts, embeddings)
    db.save_local(DB_FAISS_PATH)

if __name__ == "__main__":
    create_vector_db()


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10

In [5]:
# model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
model_name_or_path = "TechxGenus/Meta-Llama-3-8B-GPTQ"
model_basename = "model"
import torch
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
# model = AutoGPTQForCausalLM.from_quantized(
#     model_name_or_path,
#     revision="gptq-4bit-128g-actorder_True",
#     model_basename=model_basename,
#     use_safetensors=True,
#     trust_remote_code=True,
#     inject_fused_attention=False,
#     device=DEVICE,
#     quantize_config=None,
# )
model = AutoGPTQForCausalLM.from_quantized(
    model_name_or_path,
    model_basename=model_basename,
    use_safetensors=True,
    trust_remote_code=True,
    inject_fused_attention=False,
    device=DEVICE,
    quantize_config=None,
)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer,
)
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
WARNING - Exllamav2 kernel is not installed, reset disable_exllamav2 to True. This may because you installed auto_gptq using a pre-build wheel on Windows, in which exllama_kernels are not compiled. To use exllama_kernels to further speedup inference, you can re-install auto_gptq from source.
WARNING - CUDA kernels for auto_gptq are not installed, this will result in very slow inference speed. This may because:
1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.
1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.
WARNING - ignoring unknown parameter in quantize_config.

In [7]:
llm

HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7f3fd6905060>, model_kwargs={'temperature': 0})

In [6]:
import pandas as pd
csv_file = r'/content/colab_links.csv'
df = pd.read_csv(csv_file)

# Create a dictionary to map PDF filenames to URLs from the DataFrame
pdf_to_url = df.set_index('pdf')['url'].to_dict()
pdf_to_url

{'/content/drive/MyDrive/chatbot_data/data/Beinex_Website.pdf': 'https://beinex.com/',
 '/content/drive/MyDrive/chatbot_data/data/BUSINESS INTELLIGENCE.pdf': 'https://beinex.com/business-intelligence',
 '/content/drive/MyDrive/chatbot_data/data/Competitive Intelligence.pdf': 'https://beinex.com/competitive-intelligence/',
 '/content/drive/MyDrive/chatbot_data/data/Beinex Digital.pdf': 'https://beinex.com/beinex-digital/',
 '/content/drive/MyDrive/chatbot_data/data/AI & Automation.pdf': 'https://beinex.com/ai-ml-rpa/',
 '/content/drive/MyDrive/chatbot_data/data/Advanced Analytics.pdf': 'https://beinex.com/advanced-analytics/',
 '/content/drive/MyDrive/chatbot_data/data/About-Beinex.pdf': 'https://beinex.com/about-beinex/',
 '/content/drive/MyDrive/chatbot_data/data/Alteryx APA Platform.pdf': 'https://beinex.com/alteryx-apa-platform/',
 '/content/drive/MyDrive/chatbot_data/data/Alteryx-partner.pdf': 'https://beinex.com/alteryx-partner/',
 '/content/drive/MyDrive/chatbot_data/data/Beinex 

In [7]:
custom_template = """Use the following pieces of information to provide a concise and accurate response to the user's question.
If you do not have sufficient information to answer, please indicate that you do not know rather than guessing or providing incorrect information.
Answers should be brief and relevant, drawing from both the given context and the chat history. If applicable, consider answering questions based on the chat history as well.
No need to give unhelpful answer.
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CUSTOM_QUESTION_PROMPT = PromptTemplate.from_template(custom_template)
# embeddings = HuggingFaceInstructEmbeddings(
#     model_name="hkunlp/instructor-large", model_kwargs={"device": DEVICE}
# )
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                       model_kwargs={'device': DEVICE})
DB_FAISS_PATH='/content/vectorstore/db_faiss'
# db = FAISS.load_local(DB_FAISS_PATH, embeddings)
db = FAISS.load_local(DB_FAISS_PATH, embeddings, allow_dangerous_deserialization=True)
# def load_llm():
#     # Load the locally downloaded model here
#     llm = CTransformers(
#         model = "llama-2-7b-chat.ggmlv3.q8_0.bin",
#         model_type="llama",
#         max_new_tokens = 1024,
#         temperature = 0.9
#     )
#     return llm
llm = llm
from langchain.chains import ConversationalRetrievalChain
# from langchain.chat_models import ChatOpenAI

qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    db.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True,
    condense_question_prompt=CUSTOM_QUESTION_PROMPT,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
#output function
def final_result(query):
    result = qa_chain({'question': query, 'chat_history': history})
    history.append((query, result['answer']))
    url=[]
    for i in range(0, len(result['source_documents'])):
        metadata_dict = result['source_documents'][i].metadata
        source_value = metadata_dict['source'].split('\\')[-1]
        # print(source_value)
        if source_value in pdf_to_url:
            url.append(pdf_to_url[source_value])
    # print(url)
    result['url']=url
    result['contact']='https://beinex.com/contact-us/'
    response = {
    'question': result['question'],
    'answer': result['answer'],
    'For more information': result['url'],
    'Contact us': result['contact']
    }
    return response

In [9]:
history=[]
while True:
    query = input("What's on your mind? (Type 'exit' to end): ")
    if query.lower() == "exit":
        history.clear()
        break

    res = final_result(query)
    print(res)

What's on your mind? (Type 'exit' to end): What is beinex


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 Beinex is a company that provides digital transformation services.

{'question': 'What is beinex', 'answer': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nBeinex Digital, a part of Beinex Holdings, is a digital transformation entity with a comprehensive \nsuite of independent products focused on addressing specific business gaps, use cases, and \nneeds. It incorporates a spectrum of solutions in the domains of:  \n \n \n \nAurex  \nDECADES OF DIGITAL TRANSFORMATION IN DAYS  \nBeinex in Numbers  \n83% -Productivity surge in case of a govt. Entity  \n20 -Custom -built enterprise digital products  \n2500 -Professionals trained  \n350+  -Clients\n\nBEINEX DIGITAL  \nWhat makes us who we are.  \nBeinex Digital, a part of Beinex Holdings, is a digital transformation entity with a comprehensive suite \nof independent products focused on addressing specific business gaps, u

In [ ]:
history=[]
query="What are the services provided by beinex"
result = qa_chain({'question': query, 'chat_history': history})
history.append((query, result['answer']))
# history


In [ ]:
result['answer']

In [ ]:
final_result(result)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


What are the services provided by Beinex Digital?


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


 Based on the information given, we can infer that Beinex Digital provides a range of services related to digital transformation, including analytics capabilities, custom-built enterprise digital products, and professional training. However, without more specific information about each service, I cannot provide a definitive list of all the services offered by Beinex Digital.


{'question': {'question': 'What are the services provided by beinex',
  'chat_history': [('What are the services provided by beinex',
    ' Based on the information provided, here are the services offered by Beinex:\n\n1. Advanced Analytics\n2. AI/ML/RPA\n3. Risk and Audit Analytics\n4. Competitive Intelligence\n5. Data Governance\n6. Business Intelligence\n7. Modern Cloud Analytics\n8. Tableau Support Services\n9. Digital Transformation (through Beinex Digital)\n\nPlease note that this list may not be exhaustive as there could be other services offered by Beinex that are not mentioned in the given text.'),
   ({...},
    '  Based on the information given, we can infer that Beinex Digital provides a range of services related to digital transformation, including analytics capabilities, custom-built enterprise digital products, and professional training. However, without more specific information about each service, I cannot provide a definitive list of all the services offered by Beinex